# Pré processamento: 
Funções do arquivo pre_processing.py

In [98]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pre_processing as pp
import analysis as anl
import pca

category = 'all'

hNeg = False #if true, add negative bigrams for negative reviews
noun = True #if true, add nouns

X, Y, vocabulary = pp.bow(category, hNeg, noun)

print("Vocabulário possui " + str(len(vocabulary)) + " palavras!")

Vocabulário possui 24637 palavras!


In [99]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X2, Y2 = X[idx_perm, :], Y[idx_perm]

#X2, Y2 = X[idx_perm, :], Y[idx_perm]

pTrain = 0.8

train_index, test_index = anl.stratified_holdOut(Y, pTrain)

Xtrain, Xval = X2[train_index, :], X2[test_index, :]
Ytrain, Yval = Y2[train_index], Y2[test_index]

In [100]:
#Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
#Xval = Xval[:, index]

In [101]:
print("Número de features antes do chi-quadrado: " + str(len(vocabulary)))
print("----------------------------------------")
print("Número de features após chi-quadrado: " + str(len(new_vocabulary)))
#print(new_vocabulary)

Número de features antes do chi-quadrado: 24637
----------------------------------------
Número de features após chi-quadrado: 2916


# Decision Tree Clssifier

In [102]:
import scipy.stats as st

def entropy(Y) :
    entropia = st.entropy(Y)
    return entropia


In [103]:
def infoGain(X, Y, split_atribute) :
    totalEntropy = entropy(Y)
    valores, count = np.unique(X[:,split_atribute], return_counts = True)
    entropyPond = 0
    for i in range(len(valores)) : 
        
        myatt = entropy(np.where(X[:,split_atribute]==valores[i])[0])
        entropyPond += np.sum([(count[i]/np.sum(count)*myatt)])
      
    informationGain =  totalEntropy - entropyPond
    
    return informationGain

In [104]:
def ID3(X, originalX, Y, originalY, features, parentNodeClasse = None) :
    if (len(np.unique(Y)) <= 1) :
        return np.unique(Y)[0]
    if (len(X)==0) :
        return np.unique(originalY)[np.argmax(np.unique(originalY, return_counts = True))]
    if (len(features)==0) :
        return parentNodeClasse
    else :
        parentNodeClasse = np.unique(Y)[np.argmax(np.unique(Y,return_counts = True)[1])]
        bestGain = -330
        for f in features :
            gain_new = infoGain(X,Y, f)
            if(gain_new > bestGain) : 
                bestGain = gain_new
                bestFeature = f
                
        tree = {bestFeature:{}}
        
        #features.delete(bestFeature)
        features = [i for i in features if i != bestFeature]
        for value in np.unique(X[:,bestFeature]) :
            value = value
        
            subX_indes = np.where(X[:,bestFeature] == value)[0]
            subX = X[subX_indes]
            subY = Y[np.where(X[:,bestFeature] == value)]
            subTree = ID3(subX, X, subY, Y, features, parentNodeClasse)
            
            tree[bestFeature][value] = subTree
            
        return(tree) 
        

    

In [105]:
def predict(row, tree,default=1) :
    features = np.arange(Xtrain.shape[1])
    for f in features :
        if f in list(tree.keys()) :
            try :
                classe = tree[f][row[f]]
            except:
                return default    
            
            classe = tree[f][row[f]]
            if isinstance(classe,dict):
                return predict(row,classe)
            else:
                return classe
        

In [ ]:
def test(test, tree) :
    result = []
    for row in range(test.shape[0]) :
        t = test[row]
        result.append(predict(t, tree))
    return result    

In [73]:
print(Xtrain.shape[1])

from pprint import pprint
features_index = np.arange(Xtrain.shape[1])
#Xtrain = Xtrain.toarray()
#tree = ID3(Xtrain, Xtrain ,Ytrain, Ytrain, features_index)

#pprint(tree)
result = test(Xval, tree)
acuracia = np.sum(result==Yval)/len(Yval)
print(acuracia)

2894
0.48


In [ ]:
import k_folds as kf

#Pega todos os tipos de classes 
classes = classes = np.unique(Y)

# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X3, Y3 = X[idx_perm, :], Y[idx_perm]

# separa os dados em k folds
nFolds = 5
folds = kf.stratified_kfolds(Y3, nFolds, classes)

k = 1
resultados=[] # cria uma lista vazia para guardar os resultados obtidos em cada fold

for train_index, test_index in folds:

    print('\n-----------\n%d-fold: \n-----------\n' % (k) )

    # se train_index ou test_index forem vazios, interrompe o laco de repeticao
    if len(train_index)==0 or len(test_index)==0: 
        print('\tErro: o vetor com os indices de treinamento ou o vetor com os indices de teste esta vazio')      
        break
        
    #Xtrain, Xval = X3[train_index, :], X3[test_index, :];
    #Ytrain, Yval= Y3[train_index], Y3[test_index];

    #Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
    #Xval = Xval[:, index]
    
    #Converte matrizes esparsas para np arrays, para os cálculos da rede neural
    #Xtrain = Xtrain.toarray()
    #Xval = Xval.toarray()
    
    features_index = np.arange(Xtrain.shape[1])
    Xtrain = Xtrain.toarray()
    tree = ID3(Xtrain, Xtrain ,Ytrain, Ytrain, features_index)

    #pprint(tree)
    Ypred = test(Xval.toarray(), tree)
    auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)

    # adiciona os resultados do fold atual na lista de resultados
    resultados.append( auxResults )
    
    k = k + 1
    
kf.mediaFolds( resultados, classes )


-----------
1-fold: 
-----------



/home/sy/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:2508: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=0)
